The Zillow Home Value Index is the median Zestimate valuation for a given geographic area on a given day.  Find this value for each neigborhood.

In [31]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup #use this to parse out the XML

zillow_api_key='X1-ZWz1giq31ik6q3_7q4l5'

In [46]:
#get Denver Neighborhood Zillow z-indexes for neighborhood prices
#http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id=<ZWSID>&state=wa&city=seattle&childtype=neighborhood

base_url = 'http://www.zillow.com/webservice/GetRegionChildren.htm?'
parameters = {'zws-id': zillow_api_key, 
              'city': 'Denver',
              'state': 'Colorado',
              'childtype': 'neighborhood'
              }

response = requests.get(base_url, parameters)

In [29]:
response.url

'http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id=X1-ZWz1giq31ik6q3_7q4l5&city=Denver&state=Colorado&childtype=neighborhood'

In [48]:
from bs4 import BeautifulSoup #use this to parse out the XML
soup = BeautifulSoup(response.text, 'lxml')
main = soup.find('response')  #top level of xml gives request, message, and response - all the data is in response
main = main.find('list')  #within response we have region data (that was being searched on) and the list data that gives the actual neighborhoods

In [209]:
#in the list there are different 'region' (which are the child) for each neigborhood
neighborhood = []
house_value = []
for child in main.children:
    _neighborhood = child.find('name')
    _zindex = child.find('zindex')
    #need to do if-else in order to account for spots that have a Nonetype for the child
    if _neighborhood: #only add values if there is a neigborhood title in the child
        neighborhood.append(_neighborhood.get_text())
        if _zindex:
            house_value.append(_zindex.get_text())
        else: #some neigborhoods don't have zindex so put a blank spot for them
            house_value.append("")
        
print(neighborhood)
print(len(neighborhood))
print(house_value)
print(len(house_value))

['Montbello', 'Gateway - Green Valley Ranch', 'Hampden', 'Westwood', 'Capitol Hill', 'Hampden South', 'Mar Lee', 'Windsor', 'Five Points', 'Virginia Village', 'Washington Virginia Vale', 'Speer', 'Harvey Park', 'East Colfax', 'Congress Park', 'Sunnyside', 'Marston', 'Ruby Hill', 'Bear Valley', 'University', 'West Highland', 'North Park Hill', 'West Colfax', 'Harvey Park South', 'Lowry Field', 'Villa Park', 'South Park Hill', 'Highland', 'Cheesman Park', 'Athmar Park', 'Fort Logan', 'Hilltop', 'Northeast Park Hill', 'Berkeley', 'University Park', 'Sloan Lake', 'Elyria Swansea', 'Washington Park', 'Washington Park West', 'College View - South Platte', 'Hale', 'University Hills', 'Whittier', 'Lincoln Park', 'Montclair', 'Barnum', 'North Capitol Hill', 'Cherry Creek', 'Platt Park', 'Baker', 'Goldsmith', 'Barnum West', 'Cole', 'Belcaro', 'Clayton', 'Southmoor Park', 'Union Station', 'City Park West', 'Regis', 'Kennedy', 'Chaffee Park', 'Cory - Merrill', 'Valverde', 'Globeville', 'Wellshire'

In [210]:
df_census = pd.read_csv('raw_data\\census_neighborhood_demographics_2010.csv')
df_census.head()

,NBHD_ID,NBRHD_NAME,POPULATION_2010,HISPANIC_2010,WHITE_2010,BLACK_2010,NATIVEAM_2010,ASIAN_2010,HAWPACIS_2010,OTHER_2010,...,RENTED_AGE_25_TO_34,RENTED_AGE_35_TO_44,RENTED_AGE_45_TO_54,RENTED_AGE_55_TO_59,RENTED_AGE_60_TO_64,RENTED_AGE_65_TO_74,RENTED_AGE_75_TO_84,RENTED_AGE_85_PLUS,SHAPE_Length,SHAPE_Area
0,32,Hampden,17547,2505,11750,1963,64,726,20,38,...,1363,791,700,304,253,285,200,229,0.200466,0.000768
1,3,Baker,4879,1664,2854,144,43,52,10,7,...,486,235,231,113,64,82,23,7,0.101767,0.000406
2,70,Washington Park,6905,295,6356,28,9,119,3,13,...,329,132,70,31,36,27,6,21,0.090445,0.000411
3,13,Cherry Creek,5589,332,4896,79,17,168,4,3,...,521,261,211,79,92,161,224,207,0.063923,0.000228
4,22,Country Club,3001,94,2740,20,8,77,1,5,...,90,55,34,14,17,9,4,2,0.052865,0.000177


In [211]:
census_columns = df_census.columns
census_columns[0:20]

Index(['NBHD_ID', 'NBRHD_NAME', 'POPULATION_2010', 'HISPANIC_2010',
       'WHITE_2010', 'BLACK_2010', 'NATIVEAM_2010', 'ASIAN_2010',
       'HAWPACIS_2010', 'OTHER_2010', 'TWO_OR_MORE_2010', 'PCT_HISPANIC',
       'PCT_WHITE', 'PCT_BLACK', 'PCT_AMERIND', 'PCT_ASIAN', 'PCT_HAW_PACIS',
       'PCT_OTHER_RACE', 'PCT_TWO_OR_MORE_RACES', 'HOUSINGUNITS_2010'],
      dtype='object')

In [212]:
df_prices = pd.DataFrame({'Neighborhood': neighborhood, 'House Value': house_value})
df_prices = df_prices[['Neighborhood', 'House Value']]
df_prices.head()

,Neighborhood,House Value
0,Montbello,174000
1,Gateway - Green Valley Ranch,208600
2,Hampden,297900
3,Westwood,152600
4,Capitol Hill,197600


In [213]:
#need to rename three neighborhoods in order to merge zillow names with census data names
# zillow names:
# Gateway - Green Valley Ranch
# College View - South Platte
# Central Business District
# census names:
# CBD
# Gateway / Green Valley Ranch
# College View / South Platte
df_census.NBRHD_NAME.replace({'Gateway / Green Valley Ranch': 'Gateway - Green Valley Ranch', 
                                'College View / South Platte': 'College View - South Platte'},
                                inplace=True)
df_prices.Neighborhood.replace({'Central Business District': 'CBD'}, inplace=True)

In [214]:
print(df_census['NBRHD_NAME'])
print(df_prices['Neighborhood'])

0                         Hampden
1                           Baker
2                 Washington Park
3                    Cherry Creek
4                    Country Club
5                         Clayton
6                         Skyland
7                  City Park West
8                    Capitol Hill
9              North Capitol Hill
10                   Civic Center
11                            CBD
12                  Union Station
13                    Five Points
14                      Stapleton
15                       Highland
16                     Globeville
17                University Park
18                 Cory - Merrill
19           Washington Park West
20                          Speer
21                  Congress Park
22                           Cole
23                      Sunnyside
24                      Ruby Hill
25                     University
26                      Goldsmith
27               Virginia Village
28                            DIA
29            

In [215]:
df = pd.merge(df_prices, df_census[['NBRHD_NAME', 'POPULATION_2010', 'PCT_WHITE']], 
              how='outer', left_on='Neighborhood', right_on='NBRHD_NAME')

In [216]:
df[df.NBRHD_NAME.isnull()]

,Neighborhood,House Value,NBRHD_NAME,POPULATION_2010,PCT_WHITE


In [217]:
df[df.Neighborhood.isnull()]

,Neighborhood,House Value,NBRHD_NAME,POPULATION_2010,PCT_WHITE


In [218]:
df = df[['Neighborhood', 'House Value', 'POPULATION_2010', 'PCT_WHITE']]
df.head()

,Neighborhood,House Value,POPULATION_2010,PCT_WHITE
0,Montbello,174000,30348,8.17
1,Gateway - Green Valley Ranch,208600,29201,23.98
2,Hampden,297900,17547,66.96
3,Westwood,152600,15486,11.39
4,Capitol Hill,197600,14708,78.56


In [219]:
df.to_csv('raw_data\\Neighborhood_House_Value.csv')

In [220]:
#add-in Zillow median rent prices by neighborhood - this isn't in an API, but has downloadable csv data
#https://www.zillow.com/research/data/

In [221]:
df_rents = pd.read_csv('raw_data\\zillow_rent_index.csv')
df_rents.head()

,Date,RegionName,State,Metro,County,City,SizeRank,Zri,MoM,QoQ,YoY,ZriRecordCnt
0,5/31/2018,Northeast Dallas,TX,Dallas-Fort Worth,Dallas,Dallas,0,1739,-0.006286,-0.020833,0.013403,57050
1,5/31/2018,Paradise,NV,Las Vegas,Clark,Las Vegas,1,1277,0.005512,0.015911,0.057995,61870
2,5/31/2018,Maryvale,AZ,Phoenix,Maricopa,Phoenix,2,1138,0.007080,0.027076,0.099517,49156
3,5/31/2018,Upper West Side,NY,New York,New York,New York,3,3686,-0.003784,-0.012326,-0.022800,71051
4,5/31/2018,Sherman Oaks,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,Los Angeles,4,3594,-0.004708,-0.009372,0.007569,19605


In [222]:
df_rents_denver = df_rents[df_rents.City == 'Denver']

#rename 'Central Business District' to 'CBD', but leave the other names to match Denver city
df_rents_denver.RegionName.replace({'Central Business District': 'CBD'}, inplace=True)
df_rents_denver.head()

C:\Users\micha\Anaconda3\lib\site-packages\pandas\core\generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Date,RegionName,State,Metro,County,City,SizeRank,Zri,MoM,QoQ,YoY,ZriRecordCnt
238,5/31/2018,Montbello,CO,Denver,Denver,Denver,238,1854,-0.001616,-0.005898,0.045096,7416
337,5/31/2018,Gateway - Green Valley Ranch,CO,Denver,Denver,Denver,337,1951,-0.002556,-0.007630,0.056308,9721
602,5/31/2018,Hampden,CO,Denver,Denver,Denver,602,2138,0.004227,0.014231,0.037864,7331
703,5/31/2018,Westwood,CO,Denver,Denver,Denver,703,1675,0.006006,0.017001,0.049499,3291
813,5/31/2018,Capitol Hill,CO,Denver,Denver,Denver,813,1534,-0.001952,-0.003249,0.014550,5326


In [223]:
len(df_rents_denver)

75

In [224]:
df2 = pd.merge(df, df_rents_denver[['RegionName', 'Zri']], how='outer', left_on='Neighborhood', right_on='RegionName')
print(df2.shape)
df2.head()

(78, 6)


,Neighborhood,House Value,POPULATION_2010,PCT_WHITE,RegionName,Zri
0,Montbello,174000,30348,8.17,Montbello,1854.0
1,Gateway - Green Valley Ranch,208600,29201,23.98,Gateway - Green Valley Ranch,1951.0
2,Hampden,297900,17547,66.96,Hampden,2138.0
3,Westwood,152600,15486,11.39,Westwood,1675.0
4,Capitol Hill,197600,14708,78.56,Capitol Hill,1534.0


In [225]:
df2[df2.Neighborhood.isnull()]

,Neighborhood,House Value,POPULATION_2010,PCT_WHITE,RegionName,Zri


In [226]:
df2[df2.RegionName.isnull()]

,Neighborhood,House Value,POPULATION_2010,PCT_WHITE,RegionName,Zri
59,Kennedy,,4464,35.55,NaN,NaN
74,Sun Valley,,1448,7.67,NaN,NaN
77,Auraria,,705,78.72,NaN,NaN


In [227]:
df2 = df2[['Neighborhood', 'House Value', 'Zri', 'POPULATION_2010', 'PCT_WHITE']]
df2.rename(columns={'Zri': 'Median Rent'}, inplace=True)
df2.head()

,Neighborhood,House Value,Median Rent,POPULATION_2010,PCT_WHITE
0,Montbello,174000,1854.0,30348,8.17
1,Gateway - Green Valley Ranch,208600,1951.0,29201,23.98
2,Hampden,297900,2138.0,17547,66.96
3,Westwood,152600,1675.0,15486,11.39
4,Capitol Hill,197600,1534.0,14708,78.56


In [228]:
df2.to_csv('raw_data\\Neighborhood_House_Value_Median_Rents.csv')

In [229]:
df_region = pd.read_csv('raw_data\\neighborhood_regions.csv')
df_region.drop(['Unnamed: 2', 'Unnamed: 3'], axis='columns', inplace=True)
df_region.head()

,NEIGHBORHOOD,DENVER_REGION
0,Athmar Park,West
1,Auraria,Northwest
2,Baker,Central
3,Barnum,West
4,Barnum West,West


In [230]:
len(df_region)

78

In [236]:
df3 = pd.merge(df2, df_region, how='outer', left_on='Neighborhood', right_on='NEIGHBORHOOD')
df3.drop('NEIGHBORHOOD', axis='columns', inplace=True)
df3.head()

,Neighborhood,House Value,Median Rent,POPULATION_2010,PCT_WHITE,DENVER_REGION
0,Montbello,174000,1854.0,30348,8.17,Northeast
1,Gateway - Green Valley Ranch,208600,1951.0,29201,23.98,Northeast
2,Hampden,297900,2138.0,17547,66.96,Southeast
3,Westwood,152600,1675.0,15486,11.39,West
4,Capitol Hill,197600,1534.0,14708,78.56,Central


In [237]:
df3[df3.Neighborhood.isnull()]

,Neighborhood,House Value,Median Rent,POPULATION_2010,PCT_WHITE,DENVER_REGION


In [239]:
df3.to_csv('raw_data\\Neighborhood_House_Rents_Regions.csv')